In [1]:
import cv2
import numpy as np
from object_detection import ObjectDetection
from objects import Object

In [288]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\Jairo Enrique/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-28 Python-3.8.5 torch-1.12.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
def YOLOv5(frame):
    try:
        results = model(frame)
        return results.pandas().xyxy[0]
    except:
        pass

In [4]:
YOLOS = [ ["dnn_model/yolo-fastest-1.1.weights", "dnn_model/yolo-fastest-1.1.cfg", "dnn_model/coco.names.txt"],
        ["dnn_model/yolo-fastest-1.1-xl.weights", "dnn_model/yolo-fastest-1.1-xl.cfg", "dnn_model/coco.names.txt"],
         ["dnn_model/yolov3.weights", "dnn_model/yolov3.cfg", "dnn_model/coco.names.txt"],
        ["dnn_model/yolov4.weights", "dnn_model/yolov4.cfg", "dnn_model/classes.txt"]]

i = 3
YOLO = ObjectDetection(weights_path = YOLOS[i][0], 
                       cfg_path = YOLOS[i][1],
                      class_path = YOLOS[i][2])

Loading Object Detection
Running opencv dnn with YOLOv4


In [289]:
def loadClassesNames():
        with open('dnn_model/classes.txt') as f:
            Objects = [line.replace('\n','') for line in f.readlines()]
        f.close()
        return Objects
    
class Object():
    All = {}
    LastID = 0
    Threshold = 500
    MaxFramesWithOutNoMove = 200
    TemporalStep = 1
    dt = 1
    WhiteList = ["person", "car", "truck"]
    ClassesNames = loadClassesNames()
    Salieron = 0
    Entraron = 0
    
    @classmethod
    def create(cls, name, PosIni, rectangle):
        """
        if ID in cls.All.keys():
            raise IndexError(f"No puede existir dos ID's iguales ID: {ID}")
        
        if len(list(cls.All.keys())) >1 and ID != np.max(list(cls.All.keys())) + 1:
            txtErr = f"Los ID deben de ser consecutivos ID: {ID}"
            txtErr += f"\nSiguiente ID: {np.max(list(cls.All.keys())) + 1}"
            raise IndexError(txtErr)
        """
        object_ = Object(name, PosIni, rectangle)
        cls.All[Object.LastID] = object_
        return object_
    
    @classmethod
    def NewFrame(cls, Coords, Classes, Rectangles):
        
        for coor, class_, rectangle in zip(Coords, Classes, Rectangles):
            
            min_dis = 100
            min_dis_ID = None
            for ID in Object.All.keys():
                d = Object.All[ID].getDistance(coor)
                if d < Object.Threshold and d < min_dis and Object.All[ID].Life == True:
                    min_dis = d
                    min_dis_ID = ID
            # Si está más cerca de un objeto, se actualiza su posición
            if min_dis_ID != None and Object.areEqual(Object.All[min_dis_ID].name, Object.ClassesNames[class_]):
                Object.All[min_dis_ID].ActPosition(coor)
                Object.All[min_dis_ID].Rectangle = rectangle
                Object.All[min_dis_ID].LastTemporalStep = Object.TemporalStep
            #Si no, es un objeto nuevo
            else:
                Object.create(class_, coor, rectangle)
                Object.All[Object.LastID].LastTemporalStep = Object.TemporalStep
        
        
    
    @classmethod
    def areEqual(cls, Name1, Name2):
        D = {"car": "car", "truck": "car",  "person": "person"}
        return D[Name1] == D[Name2]
        
    @classmethod
    def ShowAll(cls, onlyALives = False):
        for k in Object.All.keys():
            if onlyALives:
                if Object.All[k].Life == False:
                    continue
                    
            print(" ================================== ")
            print(f"ID: {Object.All[k].ID}")
            print(f"Tipo: {Object.All[k].name}")
            print(f"Path: {Object.All[k].path}")
            print(f"Pos: {Object.All[k].getPosition()}")
            print(" ================================== ")
            print()
            
    @classmethod
    def checkDeath(cls):
        for ID in Object.All.keys():
            Diff = Object.TemporalStep - Object.All[ID].LastTemporalStep
            if Object.All[ID].Life and Diff > 4*Object.dt:
                Object.All[ID].Life = False
                Object.All[ID].checkMove()
                #print(f"Murió ID = {ID}")
                
    @classmethod
    def PutID(cls, frame):
        onlyALives = True
        for k in Object.All.keys():
            if onlyALives and Object.All[k].Life == False:
                continue
            Diff = Object.TemporalStep - Object.All[k].LastTemporalStep
            Object.All[k].stimatePosition()
            if Diff > 2*Object.dt:
                continue
            
            x, y = Object.All[k].getPosition()
            #print(x,y)
            cv2.putText(frame, str(Object.All[k].ID), (x ,y-7), 0,1, (0,0,255), 2)
            xmin, ymin, xmax, ymax = Object.All[k].Rectangle
            cv2.rectangle(roi, (round(xmin), round(ymin)), (round(xmax), round(ymax)), (255,0,0), 2)
            cv2.circle(roi, (x,y), 5, (0,0,255), -1 )
            
            
        
    @classmethod
    def reset(cls):
        cls.All = {}
        cls.LastID = 0
        cls.TemporalStep = 1
        Object.Salieron = 0
        Object.Entraron = 0
        
    @classmethod
    def step(cls, roi):
        Object.TemporalStep += 1
        Object.checkDeath()
        Object.PutID(roi)
        #TODO
    
    def __init__(self, name, PosIni, Rectangle):
        self.name = Object.ClassesNames[name]
        self.PosIni = PosIni
        self.Life = True
        self.LastTemporalStep = Object.TemporalStep
        self.ID = Object.LastID
        self.path = [PosIni]
        self.pathTemporal = [Object.TemporalStep]
        self.Rectangle = Rectangle
        self.StimatedPath = [PosIni]
        self.StimatedpathTemporal = [Object.TemporalStep]
        
        Object.LastID += 1
        
    def ActPosition(self, ActPos):
        self.path.append(ActPos)
        self.pathTemporal.append(Object.TemporalStep)
        
        self.StimatedpathTemporal = self.pathTemporal.copy()
        self.StimatedPath = self.path.copy() 
        
    def stimatePosition(self):
        #just one at time
        temp = Object.TemporalStep - self.LastTemporalStep
        dt = Object.dt# if temp <= Object.dt else  temp*2
        
        #print()
        #print(f"{self.ID} - {self.name} ")
        #print(f"Actual Position: {self.getPosition()}")
        NP = np.array(self.StimatedPath[-3:])
        
        NPT = self.StimatedpathTemporal[-3:]
        
        NP.T[0] # en x
        NP.T[1] # en y
        AceY = 0
        AceX = 0
        
        VelX = 0
        VelY = 0
        
        for k in range(NP.shape[0]-2):
            VelX1 = (NP.T[0][k + 1] - NP.T[0][k]) / max((NPT[k+1] - NPT[k]), 1)
            VelY1 = (NP.T[1][k + 1] - NP.T[1][k]) / max((NPT[k+1] - NPT[k]), 1)
            
            VelX2 = (NP.T[0][k + 2] - NP.T[0][k + 1]) / max((NPT[k+2] - NPT[k+1]), 1)
            VelY2 = (NP.T[1][k + 2] - NP.T[1][k + 1]) / max((NPT[k+2] - NPT[k+1]),1)
            
            AceX += (VelX2 - VelX1) / max((NPT[k+2] - NPT[k]), 1)
            AceY += (VelY2 - VelY1) / max((NPT[k+2] - NPT[k]), 1)
            
            VelX += VelX1
            VelY += VelY1

        if NP.shape[0]-2 > 0:
            
            VelX /= (NP.shape[0]-2)
            VelY /= (NP.shape[0]-2)
            AceX /= (NP.shape[0]-2)
            AceX /= (NP.shape[0]-2)
            
            deltaX = VelX1  + 0.5 * AceX
            deltaY = VelY1  + 0.5 * AceY
            
            """
            print(f"Real path: {self.path}")
            print(f"Estimated path: {self.StimatedPath}")
            print(f"Cambio en X: {deltaX} Cambio en Y: {deltaY}") #Aquí
            print(f"Velx: {VelX} Vely: {VelY}") #Aquí
            print(f"Ac: {AceX} Ac: {AceY}") #Aquí

            print(f"aqui {self.getPosition()[0]} , {self.getPosition()[1]}")
            print(f"aqui {self.getPosition()[0] + deltaX} , {self.getPosition()[1] + deltaY}")
            """
            
            Nx = int(self.StimatedPath[-1][0] + deltaX)
            Nx = Nx if Nx > 0 else 0
            
            Ny = int(self.StimatedPath[-1][1] + deltaY)
            Ny = Ny if Ny > 0 else 0
            """
            print(f"New {Nx}, {Ny}")
            
            print(f"Rectangle: {self.Rectangle}")
            """
            self.Rectangle = [self.Rectangle[0] + deltaX, self.Rectangle[1] + deltaY, 
                              self.Rectangle[2] + deltaX, self.Rectangle[3] + deltaY,]
            #print(f"New Rectangle: {self.Rectangle}")
            self.StimatedPath.append([Nx, Ny])
            self.StimatedpathTemporal.append( self.StimatedpathTemporal[-1] + 1)
        
    def getPosition(self):
        return self.StimatedPath[-1]
        #return self.path[-1]
    
    def checkMove(self):
        """
        Si un objeto permanece inmovil por más de n frames,
        se considera como muerto
        """
        if self.name != 'car' or np.all(np.std(np.array(self.path), axis=0) < 3):
            return
        Move = self.path[-1][1] - self.path[0][1]
        
        if Move < 0: #entró
            Object.Salieron += 1
        else:
            Object.Entraron += 1
        print("-----------------------")
        print(f"Move in {self.ID}")
        print(f"Han salido {Object.Salieron}")
        print(f"Han entrado {Object.Entraron}")
        print("-----------------------")
        print()
        #if len(self.path) > 10:
    
    def getDistance(self, Pos):
        return np.sqrt((Pos[0] - self.getPosition()[0])**2 + (Pos[1] - self.getPosition()[1])**2)



In [295]:
cap = cv2.VideoCapture('videos/short.mp4')
nFrame = -1
Object.reset()
Object.dt = 4

while True:
    nFrame += 1
    
    ret, frame = cap.read()
    if not ret:
        break
    
    if nFrame < 100:
        continue
    #print(f"=========== {nFrame} ===========")
    key = cv2.waitKey(1)
    roi = frame[650:, :450]
    
    #"""
    
    """
    (class_ids, scores, boxes) = YOLO.detect(roi)

    for box, class_id in zip(boxes, class_ids):
        if Object.ClassesNames[class_id] not in Object.WhiteList:
            continue
        (x, y, w, h) = box
        cv2.rectangle(roi, (x,y), (x + w, y + h), (255,0,0),2)
        cx = int((x + x + w)/2)
        cy = int((y + y + h)/2)
        cv2.circle(roi, (cx,cy), 5, (0,0,255), -1 )


        Object.NewFrame([[cx,cy]], [class_id])
    """
    if nFrame % Object.dt == 0:
        Results = YOLOv5(roi)
        for r in range(Results.shape[0]):
            #print(Results.iloc[i,:])
            xmin, ymin, xmax, ymax, _, class_id, name = [Results.iloc[r,i] for i in range(7)]
            if Object.ClassesNames[int(class_id)] not in Object.WhiteList:
                continue
            #cv2.rectangle(roi, (round(xmin), round(ymin)), (round(xmax), round(ymax)), (255,0,0), 2)
            cx = int((xmax + xmin)/2)
            cy = int((ymax + ymin)/2)
            #cv2.circle(roi, (cx,cy), 5, (0,0,255), -1 )

            Object.NewFrame([[cx,cy]], [class_id], [[xmin, ymin, xmax, ymax]])
    #if nFrame% 5 == 0:
    #    Object.ShowAll(onlyALives = True)
    #"""
    
    Object.step(roi)
    
    cv2.imshow("Frame", frame)
    cv2.imshow("Roi", roi)
    if key == 27:
        break  
        
cap.release()
cv2.destroyAllWindows()

-----------------------
Move in 0
Han salido 1
Han entrado 0
-----------------------

-----------------------
Move in 1
Han salido 2
Han entrado 0
-----------------------



In [231]:
np.all(np.std(np.array(Object.All[10].path), axis=0) > 3)

False

In [267]:
Object.All[3].path

[[481, 53]]

In [56]:
# Para una nueva versión, eliminarlos en lugar de matarlos en código
# del Object.All[ID]
# Pero antes revisar a donde se han ido
# detección cada n frames
xmin, ymin, xmax, ymax, _, class_id, name = [Results.iloc[0,i] for i in range(7)]
class_id

2

In [59]:
print(xmin, ymin, xmax, ymax, class_id, name)

150.32098388671875 552.5122680664062 268.04498291015625 685.5360717773438 2 car


In [61]:
(xmin, ymin)

(150.32098388671875, 552.5122680664062)

In [62]:
#Extrapolar los resultados para simular que se sigue el objeto al dibujar la cajita cada posición

(268.04498291015625, 685.5360717773438)